# May have found a simpler TimeSeries example

https://www.kaggle.com/code/iamleonie/intro-to-time-series-forecasting

In [29]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv('data.txt', delim_whitespace=True)
data['Timestamp'] = data['Timestamp'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d:%H:%M:%S.%f'))
#data['Timestamp'] = data['Timestamp'].apply(lambda x: pd.Timestamp.timestamp(x))
data.set_index('Timestamp', inplace=True)
data.drop('Source', axis=1, inplace=True)

for c in data.columns:
    data[c].replace(to_replace=-99999.000000, value=None, inplace=True)
    data[c].fillna(value=data[c].mean(), inplace=True)


/tmp/ipykernel_124195/2000409010.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('data.txt', delim_whitespace=True)
/tmp/ipykernel_124195/2000409010.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[c].replace(to_replace=-99999.000000, value=None, inplace=True)
/tmp/ipykernel_124195/2000409010.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will chan

In [30]:
temp_med = data.loc[:, ['Temp-med']]
temp_med

,Temp-med
Timestamp,
2025-03-13 00:00:00,136135.0
2025-03-13 00:05:00,127127.0
2025-03-13 00:10:00,118924.0
2025-03-13 00:15:00,176311.0
2025-03-13 00:20:00,246684.0
...,...
2025-03-19 18:10:00,250832.0
2025-03-19 18:15:00,249696.0
2025-03-19 18:20:00,237126.0


In [14]:
data

,Bt-med,Bt-min,Bt-max,Bx-med,Bx-min,Bx-max,By-med,By-min,By-max,Bz-med,...,Theta-max,Dens-med,Dens-min,Dens-max,Speed-med,Speed-min,Speed-max,Temp-med,Temp-min,Temp-max
Timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-13:00:00:00,7.21,7.03,7.26,2.6,1.66,3.09,-0.74,-2.51,0.94,-6.64,...,-56.57,0.18,0.11,0.26,555.24,547.02,561.65,136135,121117,153154
2025-03-13:00:05:00,6.98,6.95,7.03,2.03,0.53,2.41,-0.43,-0.83,-0.3,-6.68,...,-69.39,0.16,0.1,0.39,558.27,551.99,567.91,127127,92475,195859
2025-03-13:00:10:00,6.64,6.26,6.82,3.81,2.98,4.89,-0.55,-1.09,-0.21,-5.37,...,-42.48,0.11,0.1,0.19,536.44,520.12,553.48,118924,85942,154367
2025-03-13:00:15:00,6.27,5.25,6.51,3.85,2.89,4.99,-3.39,-3.59,0.28,-2.4,...,-16.41,0.1,0.1,0.23,548.7,533.3,553.31,176311,92195,243251
2025-03-13:00:20:00,6.55,6.5,6.73,5.86,5.55,6.63,-2.4,-3.07,-1.03,1.04,...,23.18,0.1,0.1,0.1,534.14,499.73,539.49,246684,60255,269295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-19:18:10:00,6.45,6.25,6.79,5.12,2.16,5.48,-3.51,-4.62,-2.61,-2.28,...,7.66,12.27,12.1,12.6,501.6,500.1,506.4,250832,250094,263787
2025-03-19:18:15:00,7.08,6.68,7.22,5.77,4.41,6.31,-1.22,-2.0,0.52,-3.26,...,-26.12,12.0,11.6,12.46,500.8,499.1,503.1,249696,241925,257422
2025-03-19:18:20:00,7.59,7.5,7.63,4.5,4.33,4.89,-1.47,-2.48,-1.32,-5.7,...,-47.91,11.81,11.56,12.25,500.7,499.2,504.0,237126,223791,245862
